In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import pickle
import pandas as pd
import itertools
from scipy import optimize
import xarray as xr

#For local jum002 HPC setup
import sys
sys.path.append("/home/jum002/store5/repo/smrt_fork/smrt")
sys.path.append("/home/jum002/store5/repo/snowmicropyn")

#local import
import akross_common_functions as AK
from smrt.permittivity.saline_snow import saline_snow_permittivity_scharien_with_stogryn95 as ssp
from smrt import make_model
from smrt.inputs import altimeter_list
from smrt.emmodel.iba import derived_IBA

from snowmicropyn import density_ssa, profile

#turn off warning
import warnings
warnings.simplefilter('ignore')
pd.set_option('mode.chained_assignment',None)

In [7]:
salt_eureka = pd.read_excel('/home/jum002/store5/data/AKROSS_data/Alert-Eureka/SMP/Eureka2016_Salinity.xlsx', sheet_name='snow', skiprows=1)
temp_eureka = pd.read_excel('/home/jum002/store5/data/AKROSS_data/Alert-Eureka/SMP/Eureka2016_Temp.xlsx', skiprows=1)

salt_alert = pd.read_excel('/home/jum002/store5/data/AKROSS_data/Alert-Eureka/SMP/Alert2017_Salinity.xlsx', skiprows=1)
temp_alert = pd.read_excel('/home/jum002/store5/data/AKROSS_data/Alert-Eureka//SMP/Alert2017_Temp.xlsx', skiprows=1)

In [8]:
def get_prop(df, colnames):
    df_prop = pd.concat([df[['Top_height (cm)']] , df[colnames].mean(axis = 1)], axis = 1).dropna()
    df_prop = df_prop.rename(columns={'Top_height (cm)' : 'height', 0: 'param'})
    return df_prop

In [9]:
salt_eureka_1 = get_prop(salt_eureka, ['Eureka_1 Pit 1', 'Eureka_1 Pit 2', 'Eureka_1 Pit 3'])
salt_eureka_2 = get_prop(salt_eureka, ['Eureka_2 Pit 1', 'Eureka_2 Pit 2', 'Eureka_2 Pit 3'])
salt_eureka_3 = get_prop(salt_eureka, ['Eureka_3 Pit 1', 'Eureka_3 Pit 2', 'Test Eureka_3 Pit 3'])
salt_eureka_4 = get_prop(salt_eureka, ['Eureka_4 Pit 1', 'Eureka_4 Pit 2', 'Eureka_4 Pit 3'])
salt_eureka_5 = get_prop(salt_eureka, ['Eureka_5 Pit 1', 'Eureka_5 Pit 2', 'Eureka_5 Pit 3'])
salt_eureka_6 = get_prop(salt_eureka, ['Eureka_6 Pit 1', 'Test Eureka_6 Pit 2', 'Eureka_6 Pit 3'])
salt_eureka_7 = get_prop(salt_eureka, ['Test Eureka_7 Pit 1', 'Eureka_7 Pit 2', 'Test Eureka_7 Pit 3'])
salt_eureka_8 = get_prop(salt_eureka, ['Eureka_8 Pit 1', 'Eureka_8 Pit 2'])

In [10]:
temp_eureka_1 = get_prop(temp_eureka, ['Eureka_1 Pit 1', 'Eureka_1 Pit 2', 'Eureka_1 Pit 3'])
temp_eureka_2 = get_prop(temp_eureka, ['Eureka_2 Pit 1', 'Eureka_2 Pit 2', 'Eureka_2 Pit 3'])
temp_eureka_3 = get_prop(temp_eureka, ['Eureka_3 Pit 1', 'Eureka_3 Pit 2', 'Test Eureka_3 Pit 3'])
temp_eureka_4 = get_prop(temp_eureka, ['Eureka_4 Pit 1', 'Eureka_4 Pit 2', 'Eureka_4 Pit 3'])
temp_eureka_5 = get_prop(temp_eureka, ['Eureka_5 Pit 1', 'Eureka_5 Pit 2', 'Eureka_5 Pit 3'])
temp_eureka_6 = get_prop(temp_eureka, ['Eureka_6 Pit 1', 'Test Eureka_6 Pit 2', 'Eureka_6 Pit 3'])
temp_eureka_7 = get_prop(temp_eureka, ['Test Eureka_7 Pit 1', 'Eureka_7 Pit 2', 'Test Eureka_7 Pit 3'])
temp_eureka_8 = get_prop(temp_eureka, ['Eureka_8 Pit 1', 'Eureka_8 Pit 2'])

In [11]:
salt_alert_1 = get_prop(salt_alert, ['Alert_1 Pit 1'])
salt_alert_2 = get_prop(salt_alert, ['Alert_2 Pit 1'])
salt_alert_3 = get_prop(salt_alert, ['Alert_3 Pit 1'])
salt_alert_4 = get_prop(salt_alert, ['Alert_4 Pit 1'])
salt_alert_5 = get_prop(salt_alert, ['Alert_5 Pit 1'])
salt_alert_6 = get_prop(salt_alert, ['Alert_6 Pit 1'])
salt_alert_8 = get_prop(salt_alert, ['Alert_8 Pit 1'])
salt_alert_9 = get_prop(salt_alert, ['Alert_9'])
salt_alert_10 = get_prop(salt_alert, ['Alert_10'])

In [12]:
temp_alert_1 = get_prop(temp_alert, ['Alert_1 Pit 1'])
temp_alert_2 = get_prop(temp_alert, ['Alert_2 Pit 1'])
temp_alert_3 = get_prop(temp_alert, ['Alert_3 Pit 1'])
temp_alert_4 = get_prop(temp_alert, ['Alert_4 Pit 1'])
temp_alert_5 = get_prop(temp_alert, ['Alert_5 Pit 1'])
temp_alert_6 = get_prop(temp_alert, ['Alert_6 Pit 1'])
temp_alert_8 = get_prop(temp_alert, ['Alert_8 Pit 1'])
temp_alert_9 = get_prop(temp_alert, ['Alert_9'])
temp_alert_10 = get_prop(temp_alert, ['Alert_10'])

In [13]:
salt_dict = {'Eureka_1' : salt_eureka_1,
            'Eureka_2' : salt_eureka_2,
            'Eureka_3' : salt_eureka_3,
            'Eureka_4' : salt_eureka_4,
            'Eureka_5' : salt_eureka_5,
            'Eureka_6' : salt_eureka_6,
            'Eureka_7' : salt_eureka_7,
            'Eureka_8' : salt_eureka_8,
            'Alert_1' : salt_alert_1,
            'Alert_2' : salt_alert_2,
            'Alert_3' : salt_alert_3,
            'Alert_4' : salt_alert_4,
            'Alert_5' : salt_alert_5,
            'Alert_6' : salt_alert_6,
            'Alert_8' : salt_alert_8,
            'Alert_9' : temp_alert_9,
            'Alert_10' : temp_alert_10}

temp_dict = {'Eureka_1' : temp_eureka_1,
            'Eureka_2' : temp_eureka_2,
            'Eureka_3' : temp_eureka_3,
            'Eureka_4' : temp_eureka_4,
            'Eureka_5' : temp_eureka_5,
            'Eureka_6' : temp_eureka_6,
            'Eureka_7' : temp_eureka_7,
            'Eureka_8' : temp_eureka_8,
            'Alert_1' : temp_alert_1,
            'Alert_2' : temp_alert_2,
            'Alert_3' : temp_alert_3,
            'Alert_4' : temp_alert_4,
            'Alert_5' : temp_alert_5,
            'Alert_6' : temp_alert_6,
            'Alert_8' : temp_alert_8,
            'Alert_9' : temp_alert_9,
            'Alert_10' : temp_alert_10}


In [14]:
with open('smrt_in-out/alert_eureka_salt_temp_dict', 'wb') as pickle_file:
    pickle.dump(salt_dict, pickle_file)